<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment20/assignment20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing all python packages

In [0]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar100
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

Using TensorFlow backend.


In [0]:
# Hyperparameters

BATCH_SIZE = 32 #@param {type:"number"}
EPOCHS = 30 #@param {type:"number"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}


In [0]:
#Configuration

NUM_CLASSES = 100 
IMG_SHAPE=[32, 32, 3]
NUM_TRAIN_IMG = 50000
NUM_TEST_IMG = 10000
BATCHES_PER_EPOCH = NUM_TRAIN_IMG//BATCH_SIZE
VALIDATION_STEPS = NUM_TEST_IMG//BATCH_SIZE

In [6]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

169009152/169001437 [==============================] - 2s 0us/step


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 1s 0us/step


In [8]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(100, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=BATCH_SIZE, validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 447s 9ms/sample - loss: 0.0550 - acc: 0.9900 - val_loss: 0.0482 - val_acc: 0.9900
Epoch 2/5
50000/50000 [==============================] - 431s 9ms/sample - loss: 0.0510 - acc: 0.9900 - val_loss: 0.0445 - val_acc: 0.9902
Epoch 3/5
50000/50000 [==============================] - 431s 9ms/sample - loss: 0.0482 - acc: 0.9900 - val_loss: 0.0413 - val_acc: 0.9905
Epoch 4/5
50000/50000 [==============================] - 432s 9ms/sample - loss: 0.0459 - acc: 0.9901 - val_loss: 0.0382 - val_acc: 0.9908
Epoch 5/5
50000/50000 [==============================] - 432s 9ms/sample - loss: 0.0439 - acc: 0.9901 - val_loss: 0.0369 - val_acc: 0.9910
